In [1]:
import os
print(os.getcwd())
#change working directory to the location of the data file
os.chdir('/mnt/d/Sajjad/08-2023/Python Code/Text Analytics - Spring 2024/')
print(os.getcwd())

/mnt/c/Users/sahaider/AppData/Local/Programs/Microsoft VS Code
/mnt/d/Sajjad/08-2023/Python Code/Text Analytics - Spring 2024


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.generation import GenerationConfig
import warnings
warnings.filterwarnings("ignore")
import textwrap

In [20]:
# Note: The default behavior now has injection attack prevention off.
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen-1_8B-Chat", trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen-1_8B-Chat-Int4", #"Qwen/Qwen-7B-Chat-Int4",
    device_map="auto",
    trust_remote_code=True
).eval()


2024-02-01 09:16:53.532373: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-01 09:16:53.745456: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-01 09:16:53.745493: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-01 09:16:53.746309: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-01 09:16:53.843180: I tensorflow/core/platform/cpu_feature_g

Solving class example using Qwen LLM

In [21]:
prompt = f"""
Classify the last sentence using the following examples.

<sentence>: it is nice weather 
<sentiment>: positive
<sentence>: it is cold 
<sentiment>: negative
<sentence>: it is not very cold 
<sentiment>: positive
<sentence>: not nice 
<sentiment>: negative
<sentence>: very cold 
<sentiment>: negative
<sentence>: cold 
<sentiment>: negative
<sentence>: very nice weather 
<sentiment>: positive
<sentence>: it is not nice cold at all

"""
response, history = model.chat(tokenizer, prompt, history=None)
print(textwrap.fill(response, width=80))

The last sentence of each example is classified as:  - "not nice": negative -
"very cold": negative - "cold": negative - "not nice": negative - "not very
cold": negative - "nice weather": positive


In [24]:
from transformers import pipeline
import pandas as pd

In [26]:
sentences = ["it is nice weather ", "it is cold ", "it is not very cold ", "not nice ", "very cold ", "cold ", "very nice weather ", "it is not nice cold at all"]

In [22]:
text = "it is not nice cold at all"

Using PLM (bert-finetune-sst2)

In [25]:
classifier = pipeline("text-classification")
outputs = classifier(text)
pd.DataFrame(outputs) 

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


,label,score
0,NEGATIVE,0.999766


In [27]:
for s in sentences:
    outputs = classifier(s)
    print(s, outputs[0]['label'], outputs[0]['score'])

it is nice weather  POSITIVE 0.9998561143875122
it is cold  NEGATIVE 0.9996206760406494
it is not very cold  POSITIVE 0.9659685492515564
not nice  NEGATIVE 0.9997665286064148
very cold  NEGATIVE 0.999714195728302
cold  NEGATIVE 0.9997304081916809
very nice weather  POSITIVE 0.9998589754104614
it is not nice cold at all NEGATIVE 0.9997656941413879


Using Qwen to evaluate all the 8 comments

In [28]:
for i, s in enumerate(sentences):
    text = s
    print(i, ":", text)
    prompt = f"""
    What is the sentiment of the following text, which is delimited with triple backticks?

    Give your answer as either "positive" or "negative". If possible, also share the probability of your answer.
    Review text: '''{text}'''
    """
    response, history = model.chat(tokenizer, prompt, history=None)
    print(textwrap.fill(response, width=80))
    print("----------------------------------")

0 : it is nice weather 
Based on the given text, it is difficult to determine whether the sentiment is
positive or negative without more context. It is a simple statement about
pleasant weather, so it's unlikely to be very nuanced or expressing strong
emotions. Therefore, I would rate the sentiment as neutral and provide a
probability of 50% that my answer is correct.
----------------------------------
1 : it is cold 
Based on the given text, the sentiment can be classified as "negative." The word
"cold" typically has a negative connotation and can evoke feelings of discomfort
or inconvenience. Therefore, the probability of the sentiment being positive
would depend on other factors such as context and personal preferences.
----------------------------------
2 : it is not very cold 
The sentiment of the text is negative. The word "not very cold" indicates that
the temperature is not pleasant and may be causing discomfort or inconvenience.
The use of triple backticks also suggests that t

Loading IMDB dataset

In [3]:
# load the dataset
with open('corpus.txt','r',encoding="utf8") as f:
    document = f.readlines()
f.close()

labels, texts = [], []
for line in document:
    content = line.split()
    label = content[0]
    labels.append(label[-1])
    texts.append(" ".join(content[1:]))

print(len(labels), len(texts))

10000 10000


In [4]:
texts[0:5]

['Stuning even for the non-gamer: This sound track was beautiful! It paints the senery in your mind so well I would recomend it even to people who hate vid. game music! I have played the game Chrono Cross but out of all of the games I have ever played it has the best music! It backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras. It would impress anyone who cares to listen! ^_^',
 "The best soundtrack ever to anything.: I'm reading a lot of reviews saying that this is the best 'game soundtrack' and I figured that I'd write a review to disagree a bit. This in my opinino is Yasunori Mitsuda's ultimate masterpiece. The music is timeless and I'm been listening to it for years now and its beauty simply refuses to fade.The price tag on this is pretty staggering I must say, but if you are going to buy any cd for this much money, this is the only one that I feel would be worth every penny.",
 'Amazing!: This soundtrack is my favorite music of all

In [5]:
labels[0:10]

['2', '2', '2', '2', '2', '2', '1', '2', '2', '2']

Preparing BoW using CountVectorizer and TfidfVectorizer

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [7]:
cnt_vectorizer = CountVectorizer()
features = cnt_vectorizer.fit_transform(texts)
features_nd = features.toarray()
features_nd.shape

(10000, 31627)

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
X_train, X_test, y_train, y_test  = train_test_split(features_nd, labels, train_size=0.75,random_state=1234)

In [10]:
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [11]:
clf1 = MultinomialNB()
clf1.fit(X_train, y_train)
y_pred = clf1.predict(X_test)
accuracy_score(y_test, y_pred)

0.846

In [12]:
cnt_vectorizer = CountVectorizer(ngram_range=(1, 2),max_features=100)
features = cnt_vectorizer.fit_transform(texts)
features_nd = features.toarray()
print(features_nd.shape)
X_train, X_test, y_train, y_test  = train_test_split(features_nd, labels, train_size=0.75,random_state=1234)
clf1 = MultinomialNB()
clf1.fit(X_train, y_train)
y_pred = clf1.predict(X_test)
accuracy_score(y_test, y_pred)

(10000, 100)


0.7156

In [13]:
cnt_vectorizer = CountVectorizer(ngram_range=(1, 2),max_features=200)
features = cnt_vectorizer.fit_transform(texts)
features_nd = features.toarray()
print(features_nd.shape)
X_train, X_test, y_train, y_test  = train_test_split(features_nd, labels, train_size=0.75,random_state=1234)
clf1 = MultinomialNB()
clf1.fit(X_train, y_train)
y_pred = clf1.predict(X_test)
accuracy_score(y_test, y_pred)

(10000, 200)


0.7384

In [14]:
cnt_vectorizer = CountVectorizer(ngram_range=(1, 2),max_features=500)
features = cnt_vectorizer.fit_transform(texts)
features_nd = features.toarray()
print(features_nd.shape)
X_train, X_test, y_train, y_test  = train_test_split(features_nd, labels, train_size=0.75,random_state=1234)
clf1 = MultinomialNB()
clf1.fit(X_train, y_train)
y_pred = clf1.predict(X_test)
accuracy_score(y_test, y_pred)

(10000, 500)


0.8024

In [15]:
cnt_vectorizer = CountVectorizer(ngram_range=(1, 2),max_features=2000)
features = cnt_vectorizer.fit_transform(texts)
features_nd = features.toarray()
print(features_nd.shape)
X_train, X_test, y_train, y_test  = train_test_split(features_nd, labels, train_size=0.75,random_state=1234)
clf1 = MultinomialNB()
clf1.fit(X_train, y_train)
y_pred = clf1.predict(X_test)
accuracy_score(y_test, y_pred)

(10000, 2000)


0.8264

In [16]:
cnt_vectorizer = CountVectorizer(ngram_range=(1, 5),max_features=2000)
features = cnt_vectorizer.fit_transform(texts)
features_nd = features.toarray()
print(features_nd.shape)
X_train, X_test, y_train, y_test  = train_test_split(features_nd, labels, train_size=0.75,random_state=1234)
clf1 = MultinomialNB()
clf1.fit(X_train, y_train)
y_pred = clf1.predict(X_test)
accuracy_score(y_test, y_pred)

(10000, 2000)


0.8228

In [17]:
cnt_vectorizer = CountVectorizer(ngram_range=(1, 3),max_features=5000)
features = cnt_vectorizer.fit_transform(texts)
features_nd = features.toarray()
print(features_nd.shape)
X_train, X_test, y_train, y_test  = train_test_split(features_nd, labels, train_size=0.75,random_state=1234)
clf1 = MultinomialNB()
clf1.fit(X_train, y_train)
y_pred = clf1.predict(X_test)
accuracy_score(y_test, y_pred)

(10000, 5000)


0.8432

In [18]:
tfidf_vectorizer = TfidfVectorizer()#(ngram_range=(1, 3),max_features=5000)
features = tfidf_vectorizer.fit_transform(texts)
features_nd = features.toarray()
print(features_nd.shape)
X_train, X_test, y_train, y_test  = train_test_split(features_nd, labels, train_size=0.75,random_state=1234)
clf1 = MultinomialNB()
clf1.fit(X_train, y_train)
y_pred = clf1.predict(X_test)
accuracy_score(y_test, y_pred)

(10000, 31627)


0.838

In [19]:
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 3),max_features=5000)
features = tfidf_vectorizer.fit_transform(texts)
features_nd = features.toarray()
print(features_nd.shape)
X_train, X_test, y_train, y_test  = train_test_split(features_nd, labels, train_size=0.75,random_state=1234)
clf1 = MultinomialNB()
clf1.fit(X_train, y_train)
y_pred = clf1.predict(X_test)
accuracy_score(y_test, y_pred)

(10000, 5000)


0.8456

In [ ]:
clf1 = RandomForestClassifier()
clf1.fit(X_train, y_train)
y_pred = clf1.predict(X_test)
accuracy_score(y_test, y_pred)

In [ ]:
clf1 = KNeighborsClassifier()
clf1.fit(X_train, y_train)
y_pred = clf1.predict(X_test)
accuracy_score(y_test, y_pred)

In [ ]:
X_train2, X_test2, y_train, y_test  = train_test_split(texts, labels, train_size=0.75,random_state=1234)

Evaluating all 2500 test records using PLM (bert-finetuned-sst)

This may take upto 2-3 minutes on your local machine

In [ ]:
bert_pred = []
for s in X_test2:
    outputs = classifier(s)
    bert_pred.append(outputs[0]['label'])

In [ ]:
#map 'Positive' to 2 and 'Negative' to 1 in bert_pred list 
bert_pred2 = []
for s in bert_pred:
    if s == 'POSITIVE':
        bert_pred2.append('2')
    else:
        bert_pred2.append('1')
accuracy_score(y_test, bert_pred2)

In [ ]:
# take top 100 records from X_test2
X_test2a = X_test[0:100]

Qwen and Zephyr can't be done on local machines (specially laptops) and should be done on Cloud